# Example: RDF workflow

This notebook provides insights on how the RDF is generated from the property table via the [rdf.py](../src/pyBiodatafuse/graph/rdf.py) module.

In [1]:
import os; os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
import pandas as pd
from pyBiodatafuse.graph import rdf
from pyBiodatafuse import constants

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/javi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the sample property table

In [2]:
data = pd.read_pickle('../combined_df.pkl')
data.head(3)

,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,MINERVA,WikiPathways,OpenTargets_reactome,OpenTargets_go,DISGENET_diseases,OpenTargets_diseases,OpenTargets_compounds,PubChem_assays,StringDB_ppi
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'disease_name': 'Mammary Neoplasms', 'HPO': ...",[{'disease_name': 'acute respiratory distress ...,"[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...",[{'disease_name': 'Multiple pterygium syndrome...,[{'disease_name': 'acute respiratory distress ...,"[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'disease_name': 'Muscular Dystrophy, Duchenn...",[{'disease_name': 'acute respiratory distress ...,"[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': nan, 'Ensembl': nan, 'sc..."


In [7]:
print(len(data))
data.describe()

932


,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,MINERVA,WikiPathways,OpenTargets_reactome,OpenTargets_go,DISGENET_diseases,OpenTargets_diseases,OpenTargets_compounds,PubChem_assays,StringDB_ppi
count,6,6,6,6,6,6,6,6,6,6,932,6,50,6
unique,6,1,6,1,6,6,6,6,6,6,72,6,50,6
top,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'disease_name': 'Mammary Neoplasms', 'HPO': ...","[{'disease_name': 'muscular dystrophy', 'thera...","[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
freq,1,6,1,6,1,1,1,1,1,1,499,1,1,1


### Generating RDF from table
Rows with no data will be skipped:

In [8]:
nas = data[data['identifier'].isna()]
print(f'Skipped row count: {len(nas)}')
nas.sample(3)

Skipped row count: 926


,identifier,identifier.source,target,target.source,Bgee_gene_expression_levels,MINERVA,WikiPathways,OpenTargets_reactome,OpenTargets_go,DISGENET_diseases,OpenTargets_diseases,OpenTargets_compounds,PubChem_assays,StringDB_ppi
144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'disease_name': 'muscular dystrophy', 'thera...",NaN,NaN,NaN
629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'disease_name': 'psoriasis', 'therapeutic_ar...",NaN,NaN,NaN
143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'disease_name': 'muscular dystrophy', 'thera...",NaN,NaN,NaN


In [5]:
g  = rdf.generate_rdf(data)

INFO:pyBiodatafuse.graph.rdf:Binding sio to http://semanticscience.org/resource/
INFO:pyBiodatafuse.graph.rdf:Binding hgnc to http://bio2rdf.org/hgnc:
INFO:pyBiodatafuse.graph.rdf:Binding obo to http://purl.obolibrary.org/obo/
INFO:pyBiodatafuse.graph.rdf:Binding umls to https://uts-ws.nlm.nih.gov/rest/semantic-network/2015AB/CUI/
INFO:pyBiodatafuse.graph.rdf:Binding ensembl to https://identifiers.org/ensembl:
INFO:pyBiodatafuse.graph.rdf:Binding dcat to http://www.w3.org/ns/dcat#
INFO:pyBiodatafuse.graph.rdf:Binding biodatafuse to https://biodatafuse.org/
INFO:pyBiodatafuse.graph.rdf:Binding foaf to http://xmlns.com/foaf/0.1/
INFO:pyBiodatafuse.graph.rdf:Binding skos to http://www.w3.org/2004/02/skos/core#
INFO:pyBiodatafuse.graph.rdf:Binding owl to http://www.w3.org/2002/07/owl#
INFO:pyBiodatafuse.graph.rdf:Binding rdf to http://www.w3.org/1999/02/22-rdf-syntax-ns#
INFO:pyBiodatafuse.graph.rdf:Binding rdfs to http://www.w3.org/2000/01/rdf-schema#
INFO:pyBiodatafuse.graph.rdf:Binding 

### Print out result

In [6]:
print(g.serialize(format="turtle"))

@prefix life_cycle_base_node: <https://biodatafuse.org/rdf/life_cycle> .
@prefix obo: <http://purl.obolibrary.org/obo/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sio: <http://semanticscience.org/resource/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://biodatafuse.org/rdf/experimental_process/000000/AHR> a sio:SIO_001077 ;
    sio:SIO_000229 <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0000178>,
        <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0000310>,
        <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0000948>,
        <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0000955>,
        <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0000990>,
        <https://biodatafuse.org/rdf/gene_expression_value/000000/AHR_UBERON_0001004>,
        <https://biodatafuse.org/rdf/gene_expre